# Adventurousness_customer_from_hh

This notebook outputs the adventrousness of a customer. 

<ul>
<li>The input file is a test file with three order histories from three customers.</li>
<li>The embedding of "key_ingredients", "title" and "description" is associated to a recipe id.</li>
<li>The embedding uses the function recipe2vec found in: /game/flaskapp_andrius/api/preprocesser.py</li>
<li>This will output the embedding associated to an order.</li>
<li>The adventurousness is calculated by doing:</li>
    <ul>
    <li>embedding1 - embedding2, embedding2 - embedding3 and so on down to the first order in time.</li>
    <li>This means calculating the difference from one embedding and the adjacent, then an average is taken.</li>
    </ul>
<li>Finally, an average over time of all the differences is made.</li>
</ul>

__[Link to stackoverflow to understand how differece is calculated](https://stackoverflow.com/questions/13114512/calculating-difference-between-two-rows-in-python-pandas)__ 

__[Link to recipe2vec function](https://github.com/S2DSLondon/Aug20_MindfulChef/blob/master/game/flaskapp_andrius/api/preprocesser.py)__ 

    

In [7]:
import pandas as pd
from loguru import logger

df = pd.read_csv('../data/test_order_history_3_customers.csv', sep = ";")

def replace_reciped_id_with_embedding(recipes, replacements):
    for index, item in enumerate(recipes):
        recipes[index] = replacements[int(item)]
    return recipes
        

df['delivery_date'] = pd.to_datetime(df['delivery_date'])
df = df.sort_values(['customer_id', 'delivery_date'])
df = df.astype('str')

#collapse all rows in one according to customer and delivery id
df_grp = df.groupby(['customer_id','delivery_id'])['id'].aggregate(
    lambda x: ','.join(tuple(x))).reset_index()

df_grp = df_grp.set_index('customer_id')
logger.info(f'this is the a simpler shape of the dataframe:{df_grp.head()}' )

#loading the embedding calculated previously by recipe2vec function 
df_emb = pd.read_csv('../data/embedding.csv')
dict_emb = df_emb.set_index('id').to_dict()

#creating the order history with the embedding
df_grp['order_history'] = df_grp['id'].apply(lambda x: x.split(','))
df_grp['emb_history'] = df_grp['order_history'].apply(
    lambda x: replace_reciped_id_with_embedding(x, dict_emb['embedding']))

#associating the mean of the embedding to one order
df_grp['mean_emb_for_one_order'] = df_grp['order_history'].apply(lambda x: sum(x)/len(x))
df_mean_emb_for_one_order = df_grp[['mean_emb_for_one_order']]
df_mean_emb_for_one_order = df_mean_emb_for_one_order.reset_index()

#dictionary: keys are customers_id, values are dataframe with order history
#for that particular customer id
dict_df ={customer: group for 
          customer, group in df_mean_emb_for_one_order.groupby(['customer_id'])
         }
#dictionary: keys are the customer id, values are the adventutousnesses:
dict_cust_adv = {keys :dict_df[keys]['mean_emb_for_one_order'].astype(float).diff().drop(
    dict_df[keys].index[0]).mean() for keys in dict_df.keys() }

df_adv = pd.DataFrame( pd.Series(dict_cust_adv, dtype = 'float'), columns = ['adventurousness'] )

df_adv.head()

2020-09-20 10:01:40.094 | INFO     | __main__:<module>:21 - this is the a simpler shape of the dataframe:            delivery_id           id
customer_id                         
94967           8018891  690,430,642
94967           8018892  942,910,219
94967           8018893  587,361,958
94967           8018894   24,242,133
94967           8018895  294,915,131


,adventurousness
94967,0.001371
94968,-0.004472
94969,-0.002028
